In [1]:
def intersection_search(intersection,line):
    '''Функция для парсинга цен по текущему запросу на сайте perekrestok.ru'''
    whole_name = ' '.join(line['Название'].split(','))

    
    variants_xpath = "/html/body/div[@class='xf ']/div[@class='xf-wrapper'][2]/section[@class='xf-search']/div[@class='xf-search-tabs js-tabs']/ul[@class='xf-search-tabs__block-list js-tabs__block-list']/li[@class='xf-search-tabs__block js-tabs__block js-search-products _active']/section[@class='xf-search-result']/div[@class='xf-search-result__panel']/div[@class='xf-search-result__group']/div[@class='js-catalog-wrap']/div[@class='xf-search-result__catalog']/ul[@id='catalogItems']/li[@class='js-catalog-product _additionals xf-catalog__item'][{}]"
    d = dict()
    for i in range(1,20):
        try:
            element = intersection.find_element_by_xpath(variants_xpath.format(i))
        except:
            break
        el = element.text.split('\n')[-3:-1]
        if len(el) == 3:
            shop_name = el[0]
            reg_price = el[1][:-4]
            promo_price = el[2][:-5]
        else:
            shop_name = el[0]
            reg_price = el[1][:-4]
            promo_price = 0
            
        # Словарь {'Степень соответствия названий товаров':'Цены товара'}
        d[SequenceMatcher(whole_name,shop_name)] = [shop_name,reg_price,promo_price]
        
    try:
        score = max(d.keys())
    except:
        return 0,0
    return accordance(d)
    

In [2]:
def platypus_search(platypus,line):
    '''Функция для парсинга цен по текущему запросу на сайте utkonos.ru'''
    whole_name = ' '.join(line['Название'].split(','))
    elements = platypus.find_elements_by_class_name('goods_pos_bottom')
    d = dict()
    for el in elements:
        if el.text != '':
            text = el.text.split('\n')[:-1]
            shop_name = ' '.join(text[0].split(','))
            reg_price = text[1]

            if len(text) == 3:
                promo_price = text[2]
            else:
                promo_price = 0
            d[SequenceMatcher(whole_name,shop_name)] = [shop_name,reg_price,promo_price]
    return accordance(d)

In [3]:
def ozon_search(ozon,line):
    '''Функция для парсинга цен по текущему запросу на сайте ozon.ru'''
    whole_name = ' '.join(line['Название'].split(','))
    d = dict()
    
    elements = ozon.find_elements_by_class_name('tile')
    
    # Если по нашему запросу ничего не найдено, возвращай None
    if len(elements) == 0:
        return None,None,None
    
    for el in elements:
        text = el.text.split('\n')[:-1]
        
        if 'Бестселлер' in text:
            text.remove('Бестселлер')
        if 'Нет в наличии' in text:
            text.remove('Нет в наличии')
        
        # Удаление всех чисел из списка
        text = [x for x in text if not (x.isdigit() or x[0] == '-' and x[1:].isdigit())]
        try:
            while True:
                fl = float(text[-1].replace(',','.',1))
                text = text[:-1]
        except:
            pass
        
        if text[0][-1] == '%':
            text = text[1:]
        
        if len(text) == 4:
            reg_price = text[1][:-2]
            promo_price = text[0][:-2]
            premium_price = text[2][:-12]
            shop_name = ' '.join(text[3].split(','))

        elif (len(text) == 3) and ('Premium' not in text[1]):
            reg_price = text[1][:-2]
            promo_price = text[0][:-2]
            premium_price = 0
            shop_name = ' '.join(text[2].split(','))
        elif (len(text) == 3) and ('Premium' in text[1]):
            reg_price = text[0][:-2]
            promo_price = 0
            premium_price = text[1][:-12]
            shop_name = ' '.join(text[2].split(','))
        elif len(text) == 2:
            reg_price = text[0][:-2]
            promo_price = 0
            premium_price = 0
            shop_name = ' '.join(text[1].split(','))
        
        
        d[SequenceMatcher(whole_name,shop_name)] = [shop_name,reg_price,promo_price,premium_price]
                
        
    return accordance(d,True)

In [4]:
def intersection_process(ind):
    '''Функция для создания и ввода запроса на сайте магазина Перекресток'''
    
    line = df.iloc[ind]
    
    intersection_request = ' '.join(line['Короткое название'].split(' ')[:2])
    intersection_text_field = intersection.find_element_by_xpath("/html/body/div[@class='xf ']/header[@class='header xf-header ']/div[@class='xf-header__sections']/div[@class='xf-wrapper']/div[@class='xf-header__sections-group']/div[@class='xf-header__search']/div[@class='js-header-search-block xf-full-header-search']/input[@class='xf-full-header-search__field tst_header-search js-search-input ui-autocomplete-input']")
    intersection_text_field.clear()
    intersection_text_field.send_keys(intersection_request)
    intersection_text_field.send_keys(Keys.RETURN)
    
    intersection_reg_price, intersection_promo_price = intersection_search(intersection,line)
    
    df['Перекресток регулярная цена'].iloc[ind] = intersection_reg_price
    df['Перекресток промо цена'].iloc[ind] = intersection_promo_price

In [10]:
def platypus_process(ind):
    '''Функция для создания и ввода запроса на сайте магазина Утконос'''
    line = df.iloc[ind]
    
    # Убираем ковычки и берем первое слово в качестве запроса
    req = re.sub(r'«','',line['Короткое название'])
    req = re.sub(r'»','',req).split(' ')[0]
    print(req)
    platypus_request = req
    
    platypus_text_field = platypus.find_element_by_name('query')
    platypus_text_field.clear()
    platypus_text_field.send_keys(platypus_request)
    platypus_text_field.send_keys(Keys.RETURN)
    
    platypus_reg_price, platypus_promo_price = platypus_search(platypus,line)
    
    df['Утконос регулярная цена'].iloc[ind] = platypus_reg_price
    df['Утконос промо цена'].iloc[ind] = platypus_promo_price

In [7]:
def ozon_process(ind):
    '''Функция для создания и ввода запроса на сайте магазина Озон'''
    
    line = df.iloc[ind]
    
    ozon_request = line['Короткое название']
    ozon_text_field = ozon.find_element_by_tag_name('input')
    for r in range(50):
        ozon_text_field.send_keys(Keys.BACK_SPACE)
    ozon_text_field.send_keys(ozon_request)
    ozon_text_field.send_keys(Keys.RETURN)
    time.sleep(5)
    
    ozon_reg_price,ozon_promo_price,ozon_premium_price = ozon_search(ozon,line)
    
    df['Озон регулярная цена'].iloc[ind] = ozon_reg_price
    df['Озон промо цена'].iloc[ind] = ozon_promo_price
    df['Озон цена с Premium'].iloc[ind] = ozon_premium_price